<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modeling" data-toc-modified-id="Modeling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modeling</a></span></li></ul></div>

In [15]:
# train, test = data.values[1:-328], data.values[-328:-6]

# train = np.array(np.split(train, len(train)/7))
# test = np.array(np.split(test, len(test)/7))

# train = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
# X, y = list(), list()
# i = 0
# for i in range(len(train) - window_size - h + 1):
#     x_input = train[i:i + window_size, 0]
#     x_input = x_input.reshape((len(x_input), 1))
#     X.append(x_input)
#     y.append(train[i + window_size:i + window_size + h, 0])
#     i += 1
# tr_X = np.array(X)
# tr_y = np.array(y)

# print(tr_X.shape)
# print(tr_y.shape)

# def evaluate_model(train, test, n_input):
# # 	model = build_model(train, n_input)
#     tr_X, tr_y = to_supervised(train, n_input)
#     verbose, epochs, batch_size = 0, 20, 16
#     n_timesteps, n_features, n_outputs = tr_X.shape[1], tr_X.shape[2], tr_y.shape[1]
#     tr_y = tr_y.reshape((tr_y.shape[0], tr_y.shape[1], 1))

#     model = Sequential()
#     model.add(Conv1D(filters=64, kernel_size=3, activation="relu", input_shape=(n_timesteps, n_features)))
#     model.add(Conv1D(filters=64, kernel_size=3, activation="relu"))
#     model.add(MaxPool1D(pool_size=2, strides=1))
#     model.add(Flatten())
#     model.add(RepeatVector(n_outputs)) # 펼친 벡터를 LSTM에다가 각각 입력으로 넣음.
#     model.add(LSTM(200, activation="relu", return_sequences=True))
#     model.add(Dropout(rate=0.5))
#     model.add(LSTM(200, activation="relu", return_sequences=True))
#     model.add(TimeDistributed(Dense(units=100, activation="relu"))) # LSTM의 리턴값으로 받은 시퀀스 각각에 대해 수행
#     model.add(TimeDistributed(Dense(units=1)))
#     model.compile(optimizer="adam", loss="mse", metrics=["acc"])
#     model.fit(tr_X, tr_y, epochs=epochs, batch_size=batch_size, verbose=verbose)

In [1]:
import pandas as pd
from datetime import datetime, timedelta
from google_drive_downloader import GoogleDriveDownloader as gdd
import math
import numpy as np
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold, GroupKFold, LeaveOneOut, LeaveOneGroupOut
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D, RepeatVector
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu
from tensorflow.keras.initializers import RandomNormal, glorot_uniform, he_uniform, Constant
from tensorflow.keras.models import load_model

In [22]:
conda install numpy=1.19.5


Note: you may need to restart the kernel to use updated packages.


In [2]:
np.__version__

'1.21.5'

In [4]:
# raw_data = pd.read_csv("D:/household_power_consumption.txt", sep=";")
# raw_data["datetime"] = pd.to_datetime(raw_data["Date"] + " " + raw_data["Time"])
# raw_data = raw_data.set_index(["datetime"], drop=True)

gdd.download_file_from_google_drive(file_id="122XXMOwYgMxvgAVrm_VwXnyV42IgBiiC", dest_path="D:/household_power_consumption.csv")

raw_data = pd.read_csv("D:/household_power_consumption.csv", header=0, infer_datetime_format=True, parse_dates=["datetime"], index_col=["datetime"])

data = raw_data.resample("D").sum()
data = data.iloc[1:-1]

In [ ]:
test_size = len(data)//10*2
cutoff = data.index.max() - timedelta(days=test_size)
data_tr, data_te = train_test_split(data["Global_active_power"], test_size=test_size, shuffle=False)

def blocked_cv(data, window_size, h):
    X = list()
    y = list()
    for i in range(len(data) - window_size - h + 1):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size:i + window_size + h])
    return np.array(X), np.array(y)

window_size = 14
h=7
tr_X, tr_y = blocked_cv(data_tr, window_size, h)
tr_X = tr_X[:, :, None]

print(tr_X.shape)
print(tr_y.shape)
# verbose, epochs, batch_size = 0, 20, 16
# n_timesteps, n_features, hputs = tr_X.shape[1], tr_X.shape[2], tr_y.shape[1]
# tr_y = tr_y.reshape((tr_y.shape[0], tr_y.shape[1], 1))

# Modeling

In [ ]:
inputs = Input(shape=(window_size, 1))

z = Conv1D(filters=64, kernel_size=3, activation="relu")(inputs)
z = Conv1D(filters=64, kernel_size=3, activation="relu")(z)
z = MaxPool1D(pool_size=2, strides=1)(z)
z = Flatten()(z)
print(z)
z = RepeatVector(h)(z)
print(z)
z = LSTM(units=200, activation="relu", return_sequences=True)(z)
z = Dropout(rate=0.5)(z)
z = LSTM(units=200, activation="relu", return_sequences=True)(z)
z = TimeDistributed(Dense(units=100, activation="relu"))(z)

outputs = TimeDistributed(Dense(units=1))(z)

model = Model(inputs=inputs, outputs=outputs)         

In [10]:
z

<tf.Tensor 'repeat_vector_1/Tile:0' shape=(None, 7, 576) dtype=float32>

In [ ]:
model.compile(optimizer="adam", loss="mse", metrics=["acc"])

model.fit(x=tr_X, y=tr_y, epochs=20, batch_size=16, verbose=2)

In [181]:
history = [x for x in data_tr]
predictions = list()
for i in range(len(test)):
    df = np.array(history)
    df = df.reshape((df.shape[0]*df.shape[1], df.shape[2]))
    input_x = df[-n_input:, 0]
    input_x = input_x.reshape((1, len(input_x), 1))
    yhat = model.predict(input_x, verbose=0)
    yhat = yhat[0]
    predictions.append(yhat)
    history.append(test[i, :])
predictions = np.array(predictions)

scores = list()
for i in range(test[:, :, 0].shape[1]):
    mse = mean_squared_error(test[:, :, 0][:, i], predictions[:, i])
    rmse = math.sqrt(mse)
    scores.append(rmse)
s = 0
for row in range(test[:, :, 0].shape[0]):
    for col in range(test[:, :, 0].shape[1]):
        s += (test[:, :, 0][row, col] - predictions[row, col])**2
score = math.sqrt(s / (test[:, :, 0].shape[0] * test[:, :, 0].shape[1]))


s_scores = ', '.join(['%.1f' % s for s in scores])
print('%s: [%.3f] %s' % ("lstm", score, s_scores))
    
days = ["sun", "mon", "tue", "wed", "thr", "fri", "sat"]
plt.plot(days, scores, marker="o", label="lstm");

IndexError: tuple index out of range

In [182]:
history

[3390.46,
 2203.8259999999964,
 1666.1940000000006,
 2225.747999999999,
 1723.2879999999993,
 2341.3379999999993,
 4773.385999999998,
 2550.012000000007,
 2743.1200000000053,
 3934.11,
 1528.7600000000004,
 2072.638,
 3174.3919999999966,
 2796.108000000001,
 3494.1959999999995,
 2749.004000000016,
 1269.2360000000006,
 1014.0540000000005,
 3259.4119999999907,
 2713.3639999999928,
 1508.3779999999997,
 2447.620000000002,
 2241.3599999999974,
 1869.053999999999,
 2154.799999999996,
 2255.1940000000004,
 1461.2960000000023,
 3186.813999999998,
 3009.441999999998,
 2148.677999999998,
 1686.4039999999998,
 2847.687999999999,
 1821.4620000000011,
 1480.4400000000028,
 3133.7319999999963,
 3407.261999999998,
 2180.494,
 1777.6400000000003,
 2987.853999999998,
 1596.3439999999975,
 1617.9719999999982,
 2044.0720000000008,
 3091.587999999989,
 1982.6640000000002,
 1707.7979999999984,
 2363.7520000000013,
 1824.7599999999998,
 1667.7359999999976,
 4029.721999999996,
 3595.9400000000023,
 1645.27